In [1]:
import os
import sys
from pathlib import Path
import pickle

import open3d as o3d
import numpy as np

# Blender-specific imports
import bpy
# *should* be able to import gpu for using GPUs to at least render, maybe bake physics?
# but I'm getting a warning that says:
#   WARN (bgl): source/blender/python/generic/bgl.c:2654 BPyInit_bgl: 'bgl' imported without an 
#   OpenGL backend. Please update your add-ons to use the 'gpu' module. In Blender 4.0 'bgl' will be
#   removed.
# This page documents usage of the Blender GPU module: https://docs.blender.org/api/current/gpu.html#module-gpu
# import gpu

# Imports from this repository
sys.path.append("../../")
sys.path.append("../")
from simulation.cloth_3d_util.accessor import Cloth3DCanonicalAccessor
from simulation.pipeline.smpl_simulation_pipeline import smpl_simulation_pipeline
from simulation.cloth_3d_util.util import loadInfo, quads2tris
from simulation.blender_util.physics import run_simulation
from simulation.blender_util_dylan.shape_keys import set_obj_default_shape_from_shape_key
from simulation.blender_util_dylan.modifiers import make_modifier_highest_priority
from simulation.blender_util_dylan.mesh import add_collision_plane
from simulation.pipeline.simulation_gripper_lower_onto_plane_pipeline import simulate_lowering_cloth_onto_table_full

%load_ext autoreload
%autoreload 2

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
WARN (bgl): source/blender/python/generic/bgl.c:2654 BPyInit_bgl: 'bgl' imported without an OpenGL backend. Please update your add-ons to use the 'gpu' module. In Blender 4.0 'bgl' will be removed.


In [2]:
FILE_ROOT = Path(os.getcwd())
CLOTH3D_PATH = Path(os.path.expanduser("~/DataLocker/datasets/CLOTH3D/training/"))
OUTPUT_ROOT = FILE_ROOT / ".." / "script_output" / "sim_pipeline_driver_test"

# Make the output directory if it doesn't exist.
OUTPUT_ROOT.mkdir(exist_ok=True)


PLANE_OFFSET = 0.025  # [m]

In [3]:
sample_configuration = {
    "sample_id": "00380",
    "garment_name": "Tshirt",
    "grip_vertex_idx": 0
}

smpl_simulation_duration_pair = (0, 120)

grip_lowering_args = {
    "initial_sim_end_frame": smpl_simulation_duration_pair[1],
    "start_frame": 1,
    "end_frame": 100,
    "fraction_lowered": 0.25, # Fraction of the cloth that will be lowered onto the table.
}

sample_configs = [sample_configuration]

In [4]:
accessor = Cloth3DCanonicalAccessor(CLOTH3D_PATH)

In [5]:
config = sample_configs[0]
sample_key = f"{config['sample_id']}_{config['garment_name']}_{config['grip_vertex_idx']}"
sample_dir = OUTPUT_ROOT / sample_key
# Make the output sample directory if it doesn't exist.
sample_dir.mkdir(exist_ok=True)

result_file = "simulation_result.pk"
result_path = sample_dir / result_file

# Get a dictionary containing the data for this sample garment.
sample_data = accessor.get_sample_data(**config)

# garment_info = accessor.reader.read_info(config["sample_id"])
garment_info_mat_filename = CLOTH3D_PATH / config["sample_id"] / "info.mat"
# print(garment_info_mat_filename)
garment_info = loadInfo(garment_info_mat_filename)

# if result_path.exists():
#     print(f"WARNING! Result file, '{result_path}', already exists! Skipping this run. Reading result instead")
#     result_data = pickle.load(result_path.open('rb'))
simulate_lowering_cloth_onto_table_full(config, sample_data, smpl_simulation_duration_pair, grip_lowering_args, sample_dir)

Running SMPL Simulation Pipeline
--------------------------------------------------------------------------------
bake: frame 0 :: 120
bake: frame 1 :: 120
bake: frame 2 :: 120
bake: frame 3 :: 120
bake: frame 4 :: 120
bake: frame 5 :: 120
bake: frame 6 :: 120
bake: frame 7 :: 120
bake: frame 8 :: 120
bake: frame 9 :: 120
bake: frame 10 :: 120
bake: frame 11 :: 120
bake: frame 12 :: 120
bake: frame 13 :: 120
bake: frame 14 :: 120
bake: frame 15 :: 120
bake: frame 16 :: 120
bake: frame 17 :: 120
bake: frame 18 :: 120
bake: frame 19 :: 120
bake: frame 20 :: 120
bake: frame 21 :: 120
bake: frame 22 :: 120
bake: frame 23 :: 120
bake: frame 24 :: 120
bake: frame 25 :: 120
bake: frame 26 :: 120
bake: frame 27 :: 120
bake: frame 28 :: 120
bake: frame 29 :: 120
bake: frame 30 :: 120
bake: frame 31 :: 120
bake: frame 32 :: 120
bake: frame 33 :: 120
bake: frame 34 :: 120
bake: frame 35 :: 120
bake: frame 36 :: 120
bake: frame 37 :: 120
bake: frame 38 :: 120
bake: frame 39 :: 120
bake: frame 40 :

In [5]:
SAVE_BLEND_FILES = False

# Turning this off for debugging below.
for config in sample_configs:
    sample_key = f"{config['sample_id']}_{config['garment_name']}_{config['grip_vertex_idx']}"
    sample_dir = OUTPUT_ROOT / sample_key
    # Make the output sample directory if it doesn't exist.
    sample_dir.mkdir(exist_ok=True)

    result_file = "simulation_result.pk"
    result_path = sample_dir / result_file

    # Get a dictionary containing the data for this sample garment.
    sample_data = accessor.get_sample_data(**config)

    # garment_info = accessor.reader.read_info(config["sample_id"])
    garment_info_mat_filename = CLOTH3D_PATH / config["sample_id"] / "info.mat"
    # print(garment_info_mat_filename)
    garment_info = loadInfo(garment_info_mat_filename)

    if result_path.exists():
        print(f"WARNING! Result file, '{result_path}', already exists! Skipping this run. Reading result instead")
        result_data = pickle.load(result_path.open('rb'))
        continue

    # TODO: Call the routines I just wrote. 
    # args_dict = dict()
    # args_dict.update(config)
    # args_dict.update(sample_data)
    # args_dict["simulation_duration_pair"] = (0, 120)

    # result_data = smpl_simulation_pipeline(**args_dict)

    ## Dylan added for controlling cloth via fake gripper so that we can lower it onto a plane.
    # NOTE: Can use blender_util.mesh.mesh_to_numpy(cloth_obj, global_coordinate=True?) to get 
    # cloth's mesh as np array.
    # TODO: Figure out how to change the coordinate frame. OR I could do this after the fact.

    # Get meshes from all frames of the physics as numpy arrays.
    # TODO: Figure out if I should start at frame 0 or 1.
    cloth_obj = bpy.data.objects["cloth"]
    animated_meshes = []
    for i in range(1, grip_lowering_args["end_frame"] + 1):
        bpy.context.scene.frame_set(i)
        mesh_data_i = mesh_to_numpy(cloth_obj, global_coordinate=True)
        animated_meshes.append(mesh_data_i)
    animated_meshes_np = np.stack(animated_meshes, axis=0)

    # Add the meshes from the lowering simulation to the result dictionary and save it.

    

    

    if SAVE_BLEND_FILES:
        blend_filepath = sample_dir / "full_simulation.blend"
        bpy.ops.wm.save_as_mainfile(filepath=blend_filepath.as_posix())

    print(f"Writing simulation results to '{result_path}'")
    pickle.dump(result_data, result_path.open('wb'))
    print("Done!")

bake: frame 0 :: 120
bake: frame 1 :: 120
bake: frame 2 :: 120
bake: frame 3 :: 120
bake: frame 4 :: 120
bake: frame 5 :: 120
bake: frame 6 :: 120
bake: frame 7 :: 120
bake: frame 8 :: 120
bake: frame 9 :: 120
bake: frame 10 :: 120
bake: frame 11 :: 120
bake: frame 12 :: 120
bake: frame 13 :: 120
bake: frame 14 :: 120
bake: frame 15 :: 120
bake: frame 16 :: 120
bake: frame 17 :: 120
bake: frame 18 :: 120
bake: frame 19 :: 120
bake: frame 20 :: 120
bake: frame 21 :: 120
bake: frame 22 :: 120
bake: frame 23 :: 120
bake: frame 24 :: 120
bake: frame 25 :: 120
bake: frame 26 :: 120
bake: frame 27 :: 120
bake: frame 28 :: 120
bake: frame 29 :: 120
bake: frame 30 :: 120
bake: frame 31 :: 120
bake: frame 32 :: 120
bake: frame 33 :: 120
bake: frame 34 :: 120
bake: frame 35 :: 120
bake: frame 36 :: 120
bake: frame 37 :: 120
bake: frame 38 :: 120
bake: frame 39 :: 120
bake: frame 40 :: 120
bake: frame 41 :: 120
bake: frame 42 :: 120
bake: frame 43 :: 120
bake: frame 44 :: 120
bake: frame 45 :: 12

In [6]:


# # I think this is just a numpy.arange for the vertices.
# # pc_keypoint_idx = point_cloud_group["human_nn_idx"][:].squeeze()
# pc_keypoint_idx = np.arange(num_verts)
# # keypoint_colors = np.random.random((pc_keypoint_idx.max() + 1, 3))
# keypoint_colors = np.random.random((num_verts, 3))

# # cloth_vert_human_nn_idx = misc_group['cloth_vert_human_nn_idx'][:]


# pc_rgb_color = pc_rgb.astype(np.float32)# / 255
# pc_keypoint_color = keypoint_colors[pc_keypoint_idx]

# pcd = o3d.geometry.PointCloud()
# pcd.points = o3d.utility.Vector3dVector(pc_point)
# # pcd.colors = o3d.utility.Vector3dVector(pc_rgb_color)
# # pcd.colors = o3d.utility.Vector3dVector(pc_keypoint_color)

# # mesh_verts_shifted = np.array(mesh_verts)
# # mesh_verts_shifted[:,0] += 0.5

# # mesh = o3d.geometry.TriangleMesh()
# # mesh.vertices = o3d.utility.Vector3dVector(mesh_verts_shifted)
# # mesh.triangles = o3d.utility.Vector3iVector(mesh_triangles)
# # mesh.vertex_colors = o3d.utility.Vector3dVector(keypoint_colors[cloth_vert_human_nn_idx])

# # o3d.visualization.draw_geometries([pcd, mesh], mesh_show_wireframe=True, mesh_show_back_face=True)

# # o3d.visualization.draw_geometries([pcd])

In [7]:
# ply_output_path = sample_dir / "grasped_shirt.ply"
# o3d.io.write_point_cloud(ply_output_path.as_posix(), pcd)

# Resources for Blender Simulation For Dynamcis

- https://blender.stackexchange.com/questions/146744/physics-control
  - Good explanation for control of an object in a physics simulation
- Use static dynamics type for objects that don't move but still need collision (like the plane potentially)
- Baking refers to the act of storing or caching the results of a calculation (like physics sim).
  - Generally recommended to bake before rendering.

## Temporary Prototyping

This will eventually go in the big loop we have above but I need to break this down so that I can test cell-by-cell first

In [5]:
config = sample_configs[0]

print("Clearing all objects!")
from simulation.blender_util.collection import remove_all_collections

remove_all_collections()


sample_key = f"{config['sample_id']}_{config['garment_name']}_{config['grip_vertex_idx']}"
sample_dir = OUTPUT_ROOT / sample_key
# Make the output sample directory if it doesn't exist.
sample_dir.mkdir(exist_ok=True)

result_file = "simulation_result.pk"
result_path = sample_dir / result_file

# Get a dictionary containing the data for this sample garment.
sample_data = accessor.get_sample_data(**config)

# garment_info = accessor.reader.read_info(config["sample_id"])
garment_info_mat_filename = CLOTH3D_PATH / config["sample_id"] / "info.mat"
# print(garment_info_mat_filename)
garment_info = loadInfo(garment_info_mat_filename)

if result_path.exists():
    # print(f"WARNING! Result file, '{result_path}', already exists! Skipping this run. Reading result instead")
    # result_data = pickle.load(result_path.open('rb'))
    # continue
    print("Results exist but resimulating for debug")
# else:
if True:
    args_dict = dict()
    args_dict.update(config)
    args_dict.update(sample_data)
    args_dict["simulation_duration_pair"] = (0, 120)

    result_data = smpl_simulation_pipeline(**args_dict)



Clearing all objects!
bake: frame 0 :: 120
bake: frame 1 :: 120
bake: frame 2 :: 120
bake: frame 3 :: 120
bake: frame 4 :: 120
bake: frame 5 :: 120
bake: frame 6 :: 120
bake: frame 7 :: 120
bake: frame 8 :: 120
bake: frame 9 :: 120
bake: frame 10 :: 120
bake: frame 11 :: 120
bake: frame 12 :: 120
bake: frame 13 :: 120
bake: frame 14 :: 120
bake: frame 15 :: 120
bake: frame 16 :: 120
bake: frame 17 :: 120
bake: frame 18 :: 120
bake: frame 19 :: 120
bake: frame 20 :: 120
bake: frame 21 :: 120
bake: frame 22 :: 120
bake: frame 23 :: 120
bake: frame 24 :: 120
bake: frame 25 :: 120
bake: frame 26 :: 120
bake: frame 27 :: 120
bake: frame 28 :: 120
bake: frame 29 :: 120
bake: frame 30 :: 120
bake: frame 31 :: 120
bake: frame 32 :: 120
bake: frame 33 :: 120
bake: frame 34 :: 120
bake: frame 35 :: 120
bake: frame 36 :: 120
bake: frame 37 :: 120
bake: frame 38 :: 120
bake: frame 39 :: 120
bake: frame 40 :: 120
bake: frame 41 :: 120
bake: frame 42 :: 120
bake: frame 43 :: 120
bake: frame 44 :: 12

In [31]:
print(sample_data.keys())

dict_keys(['human_verts', 'human_faces', 'human_gender', 'garment_verts', 'garment_faces', 'garment_uv_verts', 'garment_uv_faces', 'garment_fabric', 'garment_texture'])


In [6]:
## Dylan added for controlling cloth via fake gripper so that we can lower it onto a plane.
# NOTE: Can use blender_util.mesh.mesh_to_numpy(cloth_obj, global_coordinate=True?) to get 
# cloth's mesh as np array.
# TODO: Figure out how to change the coordinate frame. OR I could do this after the fact.
# TODO: Can I only simulate once by using Blender's shape keys?

# Add arguments for lowering the cloth onto the table.
grip_lowering_args = {
    "initial_sim_end_frame": 120,
    # "start_frame": 1,
    "end_frame": 100,  # Just for debugging now. Will increase duration once I know it works.
    "fraction_lowered": 0.15, # Fraction of the cloth that will be lowered 
}

cloth_obj = bpy.data.objects["cloth"]

# Cloth modifier is named 'CLOTH' and is accessed with cloth_obj.modifiers['CLOTH']

In [7]:
type(cloth_obj)

bpy_types.Object

In [8]:
# Add shape keys for the hook modifier and the cloth modifier?
# NOTE: No hook modifier is present, just the cloth modifier.

# TODO: I might have to set the cloth object as the active object
# print(bpy.context.active_object)  # This seems to indicate the cloth is the active object.

# First, set the active frame to one that is after the initial simulation.
# TODO: Update this with the args_dict value in the final big for loop.
bpy.context.scene.frame_set(grip_lowering_args["initial_sim_end_frame"] + 1)

# Then, apply the cloth modifier as a shapekey.
bpy.ops.object.modifier_apply_as_shapekey(keep_modifier=True, modifier="CLOTH")




RNA_boolean_get: OBJECT_OT_modifier_apply_as_shapekey.single_user not found.
RNA_boolean_get: OBJECT_OT_modifier_apply_as_shapekey.merge_customdata not found.


{'FINISHED'}

In [10]:
# Set the mesh's default shape to the shape of the shape key we just created.
set_obj_default_shape_from_shape_key(cloth_obj, "CLOTH", verbose=True)

In [11]:
# Save this as a checkpoint to make sure things look okay.
blend_filepath = sample_dir / "full_simulation_prototyping_lowering_checkpoint_1.blend"
bpy.ops.wm.save_as_mainfile(filepath=blend_filepath.as_posix())

Info: Total files 0 | Changed 0 | Failed 0
Info: Saved "full_simulation_prototyping_lowering_checkpoint_1.blend"


{'FINISHED'}

In [12]:
# Reset to the 1st frame of the simulation.
# TODO: Figure out if this is 1 or 0. I believe it's 0.
# bpy.context.scene.frame_set(0)
bpy.context.scene.frame_set(1)

# Delete the physics cache.
bpy.ops.ptcache.free_bake_all()

{'FINISHED'}

In [13]:
# Just checking that the "pin" VertexGroup is here.
print(cloth_obj.vertex_groups[0])

<bpy_struct, VertexGroup("pin") at 0x633abc8>


In [14]:
# Add a hook to a new empty so that we can control the cloth and lower it onto the "table"
bpy.ops.object.editmode_toggle()
bpy.ops.object.vertex_group_set_active(group="pin")
bpy.ops.object.hook_add_newob()
bpy.ops.object.editmode_toggle()

{'FINISHED'}

In [15]:
# Check that the new "Empty" object was created.
all_obj_names = [o.name for o in bpy.data.objects]
assert ("Empty" in all_obj_names)
# for obj in bpy.data.objects:
#     print(obj.name)

In [17]:
# Ensure that the cloth object is the selected one.
# print(bpy.context.active_object.name)

cloth


In [19]:
# Move the newly created hook modifier to be above the CLOTH modifier so that it takes precedent.
# This is required to be able to move the cloth like a gripper.
# bpy.ops.object.modifier_set_active(modifier="Hook-Empty")
# bpy.ops.object.modifier_move_to_index(modifier="Hook-Empty", index=0)

make_modifier_highest_priority("Hook-Empty")

In [22]:
## Add animation of the virtual gripper. We'll lower the cloth onto the table over some period of 
# time, then stay stationary for a brief period of time to let the cloth settle.

# Get the minimum Z value of all vertices.
min_z_val = result_data['cloth_state']['verts'][:, 2].min()
min_z_val_abs = np.abs(min_z_val)

# The amount we move the "gripper" (empty object) downward should be <fraction_moved> * abs(min_z_val)
gripper_dz = min_z_val_abs * grip_lowering_args["fraction_lowered"]

# Select the Empty object.
bpy.context.view_layer.objects.active = bpy.data.objects["Empty"]
print(bpy.context.active_object)

<bpy_struct, Object("Empty") at 0x7497348>


In [21]:
# Add keyframe for empty at start frame.
bpy.context.scene.frame_set(1)
empty_obj = bpy.data.objects["Empty"]
empty_obj.keyframe_insert(data_path="location", frame=1)

True

In [23]:
# Move the gripper to the final Z value at the final frame.
empty_obj_z_init = empty_obj.location[2]
bpy.context.scene.frame_set(grip_lowering_args["end_frame"])
empty_obj.location[2] = empty_obj_z_init - gripper_dz


In [24]:
# Now insert another keyframe at the final frame so that Blender will animate the movement for us.
empty_obj.keyframe_insert(data_path="location", frame=grip_lowering_args["end_frame"])

True

In [25]:
# Save this all as a .blend file so we can make sure it looks right so far.
blend_filepath = sample_dir / "full_simulation_prototyping_lowering_checkpoint_2.blend"
bpy.ops.wm.save_as_mainfile(filepath=blend_filepath.as_posix())

Info: Saved "full_simulation_prototyping_lowering_checkpoint_2.blend"


{'FINISHED'}

In [28]:
# Add a plane below the lowest vertex value from the initial simulation. This is what we'll 
# lower the mesh onto to deform it. 
# NOTE: The plane must be in the same collection as the cloth.
#       Additionally, the plane must have collision physics turned on.
add_collision_plane(min_z_val, PLANE_OFFSET)

In [29]:
# Clear the bake cache for the physics.
bpy.ops.ptcache.free_bake_all()

# Select the cloth object.
bpy.context.view_layer.objects.active = bpy.data.objects["cloth"]

# Change the physics simulation to last as long as we specify above.
cloth_modifier = bpy.context.object.modifiers["CLOTH"]
cloth_modifier.point_cache.frame_start = 0
cloth_modifier.point_cache.frame_end = grip_lowering_args["end_frame"]
cloth_modifier.collision_settings.use_collision = True
cloth_modifier.collision_settings.distance_min = 0.005


# Bake the physics
run_simulation(cloth_obj, cloth_modifier.point_cache)

bake: frame 0 :: 100
bake: frame 1 :: 100
bake: frame 2 :: 100
bake: frame 3 :: 100
bake: frame 4 :: 100
bake: frame 5 :: 100
bake: frame 6 :: 100
bake: frame 7 :: 100
bake: frame 8 :: 100
bake: frame 9 :: 100
bake: frame 10 :: 100
bake: frame 11 :: 100
bake: frame 12 :: 100
bake: frame 13 :: 100
bake: frame 14 :: 100
bake: frame 15 :: 100
bake: frame 16 :: 100
bake: frame 17 :: 100
bake: frame 18 :: 100
bake: frame 19 :: 100
bake: frame 20 :: 100
bake: frame 21 :: 100
bake: frame 22 :: 100
bake: frame 23 :: 100
bake: frame 24 :: 100
bake: frame 25 :: 100
bake: frame 26 :: 100
bake: frame 27 :: 100
bake: frame 28 :: 100
bake: frame 29 :: 100
bake: frame 30 :: 100
bake: frame 31 :: 100
bake: frame 32 :: 100
bake: frame 33 :: 100
bake: frame 34 :: 100
bake: frame 35 :: 100
bake: frame 36 :: 100
bake: frame 37 :: 100
bake: frame 38 :: 100
bake: frame 39 :: 100
bake: frame 40 :: 100
bake: frame 41 :: 100
bake: frame 42 :: 100
bake: frame 43 :: 100
bake: frame 44 :: 100
bake: frame 45 :: 10

In [29]:
# Save the final checkpoint to make sure it all looks good.
blend_filepath = sample_dir / "full_simulation_prototyping_lowering_checkpoint_3.blend"
bpy.ops.wm.save_as_mainfile(filepath=blend_filepath.as_posix())

Info: Saved "full_simulation_prototyping_lowering_checkpoint_3.blend"


{'FINISHED'}

In [30]:
















# Now bake the physics

# Get meshes from all frames of the physics as numpy arrays.
# TODO: Figure out if I should start at frame 0 or 1. I think starting from 0 is correct
# animated_meshes = []
# for i in range(grip_lowering_args["end_frame"]):
#     bpy.context.scene.frame_set(i)
#     mesh_data_i = mesh_to_numpy(cloth_obj, global_coordinate=True)
#     animated_meshes.append(mesh_data_i)
# animated_meshes_np = np.stack(animated_meshes, axis=0)

# # Add the meshes from the lowering simulation to the result dictionary and save it.





# if SAVE_BLEND_FILES:
#     blend_filepath = sample_dir / "full_simulation_prototyping_lowering.blend"
#     bpy.ops.wm.save_as_mainfile(filepath=blend_filepath.as_posix())

# print(f"Writing simulation results to '{result_path}'")
# pickle.dump(result_data, result_path.open('wb'))
# print("Done!")